# Mixer correction

Setup
-----

First, we are going to import the required packages.

In [1]:
# Import ipython widgets
import json
import math
import os

import ipywidgets as widgets
import matplotlib.pyplot
import numpy as np
import rich
import pandas as pd

# Set up the environment.
import scipy.signal
from IPython.display import display
from ipywidgets import fixed, interact, interact_manual, interactive

from qblox_instruments import Cluster, PlugAndPlay, Pulsar
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# This is the table that we want to populate
calibration = pd.DataFrame()
calibration["module"]  = []
calibration["complex_output"] = []
calibration["sequencer index"] = []
calibration["lo_freq (Hz)"] = []
calibration["if (Hz)"] = []
calibration["dc_mixer_offset_I"] = []
calibration["dc_mixer_offset_Q"] = []
calibration["mixer_amp_ratio"] = []
calibration["mixer_phase_error_deg"] = []
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg


## Create Q1ASM program to play on the device

In [3]:
#Waveform dictionary (data will hold the samples and index will be used to select the waveforms in the instrument).
waveforms = {
             "I":     {"data": [], "index": 0},
             "Q":     {"data": [], "index": 1}
            }

waveforms["I"]["data"] = [0.8]*8000
waveforms["Q"]["data"] = [0.0]*8000

#Sequence program.
prog = f"start: set_mrk 15 \n play 0,1,1000\n set_mrk 15 \n play 0,1,1000\njmp @start"

#Sequence program which stops.
# prog = """
#         move 1250000, R0
#         set_mrk    15
# start:
#         reset_ph
#         upd_param  4
#         play       0,1,4
#         wait       7996
#         loop       R0,@start
#         set_mrk    0
#         upd_param  4
#         stop
# """

#Reformat waveforms to lists if necessary.
for name in waveforms:
    if str(type(waveforms[name]["data"]).__name__) == "ndarray":
        waveforms[name]["data"] = waveforms[name]["data"].tolist()  # JSON only supports lists

#Add sequence program and waveforms to single dictionary and write to JSON file.
wave_and_prog_dict = {"waveforms": waveforms, "weights": {}, "acquisitions": {}, "program": prog}
with open("sequence.json", 'w', encoding='utf-8') as file:
    json.dump(wave_and_prog_dict, file, indent=4)
    file.close()

## Select Device

We scan for the available devices connected via ethernet using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/api_reference/pnp.html) for more info).

In [4]:
# Scan for available devices and display
with PlugAndPlay() as p:
    # get info of all devices
    device_list = p.list_devices()
    device_keys = list(device_list.keys())

# rich.print(device_list)
# create widget for names and ip addresses
connect = widgets.Dropdown(
    options=[(device_list[key]["identity"]["ip"]) for key in device_list.keys()],
    description="Select Device",
)
display(connect)

Dropdown(description='Select Device', options=('192.0.2.141', '192.0.2.72', '192.0.2.143', '192.0.2.142'), val…

In [20]:
# close all previous connections to the cluster
Cluster.close_all()

#device_name = "pingu_cluster"
device_name = "loki_cluster"
ip_address = connect.value

# connect to the cluster and reset
cluster = Cluster(device_name, ip_address)
cluster.reset()
print(f"{device_name} connected at {ip_address}")
cluster.identify()

loki_cluster connected at 192.0.2.141


## Select Module in Device

In [30]:
# Find all QRM/QCM modules
cluster.reset()
available_slots = {}
for module in cluster.modules:
    # if module is currently present in stack
    if cluster._get_modules_present(module.slot_idx):
        # check if QxM is RF or baseband
        if module.is_rf_type:
            available_slots[f"module{module.slot_idx}"] = ["QCM-RF", "QRM-RF"][
                module.is_qrm_type
            ]
        else:
            available_slots[f"module{module.slot_idx}"] = ["QCM", "QRM"][
                module.is_qrm_type
            ]

# List of all QxM modules present
connect_qxm = widgets.Dropdown(options=[key for key in available_slots.keys()])
display(connect_qxm)

Dropdown(options=('module1', 'module2', 'module3', 'module4', 'module5', 'module6', 'module7', 'module8', 'mod…

In [52]:
# Connect to the cluster QxM module
module = connect_qxm.value
print(module)
qxm = getattr(cluster, module)
print(f"{available_slots[connect_qxm.value]} connected")
print(cluster.get_system_state())

module17
QRM-RF connected
Status: OKAY, Flags: NONE, Slot flags: NONE


## Select complex output

In [53]:
outputs = set(map(lambda i: i.split("_")[0], filter(lambda k: "out" in k and (not k.startswith("_")), dir(qxm))))
select_out = widgets.Dropdown(options = [key for key in outputs])
display(select_out)

Dropdown(options=('out0',), value='out0')

In [24]:
qb_freq = np.array([3.69374	,3.34177	,3.5669	,3.30677	,3.85258	,3.16559	,3.90661	,3.22831	,3.89478	,3.31304,3.74806	,3.30409	,3.88604	,3.25294	,3.98161
])*1e9
lo = qb_freq + 100e6
lo_list = [print(i) for i in enumerate(lo)]

(0, 3793740000.0)
(1, 3441770000.0)
(2, 3666900000.0)
(3, 3406770000.0)
(4, 3952580000.0)
(5, 3265590000.0)
(6, 4006610000.0)
(7, 3328310000.0)
(8, 3994780000.0)
(9, 3413040000.0)
(10, 3848060000.0)
(11, 3404090000.0)
(12, 3986040000.0)
(13, 3352940000.0)
(14, 4081610000.0)


In [54]:
lo_frequency = float(input(f"Enter desired LO frequency (Hz) for '{select_out.value}': "))
if qxm.module_type.name == "QRM":
    # qrms only have one complex output
    qxm.out0_in0_lo_en(True)
    qxm.out0_in0_lo_freq(lo_frequency)
else:
    lo_en = getattr(qxm, f"{select_out.value}_lo_en")
    lo_freq = getattr(qxm, f"{select_out.value}_lo_freq")
    lo_en(True)
    lo_freq(lo_frequency)

Enter desired LO frequency (Hz) for 'out0':  6.74e9


## Select Sequencer in Module

In [55]:
qxm.stop_sequencer()
sequencers = { k : getattr(qxm, k) for k in filter(lambda a: a.startswith("sequencer") and (a != "sequencers"), dir(qxm)) }
select_seq = widgets.Dropdown(options = [key for key in sequencers.keys()])
display(select_seq)

Dropdown(options=('sequencer0', 'sequencer1', 'sequencer2', 'sequencer3', 'sequencer4', 'sequencer5'), value='…

## Upload program to sequencer

In [56]:
seq = sequencers[select_seq.value]
seq.nco_freq(MY_IF := float(input("ENTER IF: ")))   # using 10 MHz IF
seq.sequence(os.path.join(os.getcwd(), "sequence.json"))
seq.mod_en_awg(True)
print(MY_IF)
print(seq.name)

ENTER IF:  -100e6


-100000000.0
loki_cluster_module17_sequencer0


## Control sliders

In [57]:
rich.print(f"1.\tConnect complex output '{select_out.value}' of cluster '{device_name}' module '{module}' to a spectrum analyser.")
rich.print(f"2.\tIn the spectrum analyser, set the center frequency to {lo_frequency/1e9} GHz")
rich.print(f"3.\tAdjust the gain ratio and phase offset sliders until the LSB is gone.")
rich.print(f"4.\tAdjust the I and Q offsets until the LO peak dissapears.")
rich.print(f"5.\tWhen only RSB is visible in the spectrum analyser, save the calibration with the last cell.")

calibrated_values = [0,0,0.5,0] if seq.seq_idx == 0 else calibration.loc[len(calibration)-1][-4:].values.tolist() # storage array
#calibrated_values = [-0.004462799999999999*1000, -0.0097599*1000, 0.9824, -5.3845]

def set_offset_I(offset_I):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path0(offset_I) # qrm have only one output
    else:
        #print(f"{select_out.value}_offset_path0")
        offset_path0 = getattr(qxm, f"{select_out.value}_offset_path0")
        offset_path0(offset_I)

    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[0] = offset_I

def set_offset_Q(offset_Q):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path1(offset_Q) # qrm have only one output
    else:
        offset_path1 = getattr(qxm, f"{select_out.value}_offset_path1")
        offset_path1(offset_Q)
        
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[1] = offset_Q

def set_gain_ratio(gain_ratio):
    seq.mixer_corr_gain_ratio(gain_ratio)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[2] = gain_ratio

def set_phase_offset(phase_offset):
    seq.mixer_corr_phase_offset_degree(phase_offset)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[3] = phase_offset

interact(
    set_offset_I, offset_I=widgets.FloatSlider(min=-20.0, max=20.0, step=0.0001, value=calibrated_values[0], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_offset_Q, offset_Q=widgets.FloatSlider(min=-20.0, max=20.0, step=0.0001, value=calibrated_values[1], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_gain_ratio,
    gain_ratio=widgets.FloatSlider(min=0.9, max=1.2, step=0.0001, value=calibrated_values[2], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)
interact(
    set_phase_offset,
    phase_offset=widgets.FloatSlider(min=-40.0, max=40.0, step=0.00001, value=calibrated_values[3], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)

1.      Connect complex output 'out0' of cluster 'loki_cluster' module 'module17' to a spectrum analyser.

2.      In the spectrum analyser, set the center frequency to 6.74 GHz

3.      Adjust the gain ratio and phase offset sliders until the LSB is gone.

4.      Adjust the I and Q offsets until the LO peak dissapears.

5.      When only RSB is visible in the spectrum analyser, save the calibration with the last cell.

interactive(children=(FloatSlider(value=0.0, description='offset_I', layout=Layout(width='95%'), max=20.0, min…

interactive(children=(FloatSlider(value=0.0, description='offset_Q', layout=Layout(width='95%'), max=20.0, min…

interactive(children=(FloatSlider(value=0.9, description='gain_ratio', layout=Layout(width='95%'), max=1.2, mi…

interactive(children=(FloatSlider(value=0.0, description='phase_offset', layout=Layout(width='95%'), max=40.0,…

<function __main__.set_phase_offset(phase_offset)>

In [58]:
calibration.loc[len(calibration)] = [module, select_out.value, seq.seq_idx, lo_frequency, MY_IF, *tuple(calibrated_values)]
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg
0,module6,out0,0,3.793740e+09,-100000000.0,-8.9746,-0.0724,0.9844,-23.81146
1,module7,out0,0,3.441770e+09,-100000000.0,-10.3496,-7.2376,1.0111,-23.30484
2,module8,out0,0,3.666900e+09,-100000000.0,-10.3497,-2.4246,0.9955,-10.63916
3,module9,out0,0,3.406770e+09,-100000000.0,-10.6392,-10.1687,1.0188,-18.96232
4,module10,out0,0,3.952580e+09,-100000000.0,-5.4282,-1.0132,0.9860,-17.51481
5,module17,out0,0,6.740000e+09,-100000000.0,-7.3461,-6.3329,0.9000,0.00000


In [60]:
calibration.to_csv("mixercorrectionLokeB25QBC0.csv")

In [61]:
cluster.reset()